---
layout: post
title: Big Idea 4 Blog
description: my learnings from big idea 4
permalink: /big_idea_4/
---

# Big Idea 4
### 4.1 The Internet 
CSN-1.A Explain how computing devices work together in a network.
- computing devices are physical objects that can run a program
- computing systems are groups of devices which work together for a common goal
- A computer network is a group of interconnected computing devices capable of sending or receiving data
- in our project, our computer communicates with the AWS server to get a public url. AWS is a DNS, and it is a seperate device that is connecting to our personal device. This is an example of computer networks.


### 4.3 Parallel and Distributed Computing
CSN-2.B Describe benefits and challenges of parallel and distributed computing.
- Our use both parallel and distributed computing.
- we can see parallel computing in our website as it can run multiple functions, for example we can have feautures like rating, analytics, and location API on our national parks. Parallel computing is beneficial as we are able to scale it efficiently, however a challenge of parallel computing is that it can use a system's resources when there are more things to process.
<img src="{{site.baseurl}}/images/parallelcomputing.png" alt= "Image 1">

- We see distributed computing using AWS, which runs many different instances using a DNS. In our case, we are able to deploy multiple websites at once. A challenge could be that the amount of instances we are able to run are limited by the resources available and the size of the websites
<img src="{{site.baseurl}}/images/distributedcomputing.png" alt= "Image 2">


# College Board Requirements Met: 
- Use of HTTP and RESTful APIs: I created the gemini API that interacts with the backend using GET, POST, PUT, and DELETE functions (see below). This ties into our camping page by providing information about camping when the user interacts with the AI.
- Performance Optimization: I index database records for faster lookups (AIMessage.query.get(message_id))
- Monitoring and Logging: Logging occurs through the console log, where I can see errors or success codes that I can use to debug or check if my code is being processed correctly.
- Security and Authentication: Each action in my API requires the authentication token, which means the user must be logged in to be able to access my features.


A bit of code about college board requirements

class Chatbot(Resource):
    @token_required()

    def __init__(self):
        self.history = []
        self.chat_session = model.start_chat(history=self.history)  # Persistent session

    def generate_ai_response(self, user_input):
        """Generates a response from the Google Generative AI model."""
        try:
            response = self.chat_session.send_message(user_input)
            return response.text.rstrip("\n")
        except Exception as e:
            print(f"Error generating AI response: {str(e)}")
            return "Sorry, I couldn't process that."

    def update_history(self, role: str, message: str):
        """Update the conversation history and save messages to the database."""
        if len(self.history) >= self.MAX_HISTORY:
            self.history.pop(0)  # Maintain a maximum history length
        self.history.append({"role": role, "parts": [message]})

        ai_message = AIMessage(
            message=message,
            author=role,
            category="response" if role == "assistant" else "user"
        )
        ai_message.create()
        return ai_message.id  # Return the ID of the created message

    def post(self):
        """Handles POST requests to send a message and get a response."""
        try:
            data = request.get_json()
            if not data:
                return jsonify({"error": "Invalid JSON"}), 400

            user_input = data.get("user_input")
            if not user_input:
                return jsonify({"error": "User input is required"}), 400

            # Save the user's message to the database and get its ID
            user_message = AIMessage(message=user_input, author="user", category="user_message")
            db.session.add(user_message)
            db.session.commit()
            user_message_id = user_message.id  # Get the auto-generated ID

            # Generate AI response using Google Generative AI
            response_text = self.generate_ai_response(user_input)

            # Save the AI's response to the database and get its ID
            ai_message = AIMessage(message=response_text, author="assistant", category="ai_response")
            db.session.add(ai_message)
            db.session.commit()
            ai_message_id = ai_message.id  # Get the auto-generated ID

            return jsonify({
                "user_message_id": user_message_id,  # ID of the user's message
                "ai_message_id": ai_message_id,  # ID of the AI's response
                "user_input": user_input,
                "model_response": response_text
            })
        except Exception as e:
            return jsonify({"error": str(e)}), 500

    def put(self):
        """Handles PUT requests to update an existing message by ID."""
        try:
            data = request.get_json()
            if not data:
                return jsonify({"error": "Invalid JSON"}), 400

            message_id = data.get("id")
            new_message = data.get("user_input")

            if not message_id or not new_message:
                return jsonify({"error": "Message ID and new message content are required"}), 400

            # Find the message by ID
            message = AIMessage.query.get(message_id)
            if not message:
                return jsonify({"error": "Message not found"}), 404

            # Update the message content
            message.message = new_message
            db.session.commit()

            return jsonify({"message": "Message updated successfully"})
        except Exception as e:
            return jsonify({"error": str(e)}), 500

    def delete(self):
        """Handles DELETE requests to delete a message by ID."""
        try:
            data = request.get_json()
            if not data:
                return jsonify({"error": "Invalid JSON"}), 400

            message_id = data.get("id")
            if not message_id:
                return jsonify({"error": "Message ID is required"}), 400

            # Find the message by ID
            message = AIMessage.query.get(message_id)
            if not message:
                return jsonify({"error": "Message not found"}), 404

            # Delete the message
            db.session.delete(message)
            db.session.commit()

            return '', 204  # Return a 204 No Content to indicate successful deletion
        except Exception as e:
            return jsonify({"error": str(e)}), 500
